In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install chars2vec

     |████████████████████████████████| 8.1 MB 7.7 MB/s 
  Created wheel for chars2vec: filename=chars2vec-0.1.7-py3-none-any.whl size=8111097 sha256=8052ed9bd11ab97679bcf679f87610be2cb034e80ccc2e46395a559da2096d6a
  Stored in directory: /root/.cache/pip/wheels/6d/55/76/0797aaeb92faa4840c9e352353f741aacbe443cb00c66af1c6
Successfully built chars2vec


In [ ]:
# problem 파일 경로 Desktop/project/code_similarity/open/code

# problem 폴더 안 예제 코드 확장자명 변경
import os
from tqdm import tqdm
import re
# .txt 병합
import pandas as pd
import numpy as np
import sklearn

from keras.layers import Input
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
pwd

'/content'

In [ ]:
# 폴더 및 .py 파일 개수 확인
code_folder = "./drive/MyDrive/code_similarity/open/code"
problem_folders = os.listdir(code_folder)

#Ds.store숨김
problem_folders = [folder for folder in problem_folders if folder.startswith('problem')]

# 폴더명 정렬
problem_folders.sort(reverse=False)
len(problem_folders)

300

In [ ]:
def preprocess_script(script):
    '''
    간단한 전처리 함수
    주석 -> 삭제
    '    '-> tab 변환
    다중 개행 -> 한 번으로 변환
    '''
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        preproc_lines = []
        for line in lines:
            if line.lstrip().startswith('#'):
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')]
            line = line.replace('\n','')
            line = line.replace('    ','\t')
            if line == '':
                continue
            preproc_lines.append(line)
        preprocessed_script = '\n'.join(preproc_lines)
    return preprocessed_script

preproc_scripts = []
problem_nums = []

for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder,problem_folder))
    problem_num = scripts[7].split('_')[0] 
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preproc_scripts.append(preprocessed_script)
    problem_nums.extend([problem_num]*len(scripts))

100%|██████████| 300/300 [08:32<00:00,  1.71s/it]


In [ ]:
code_df = pd.DataFrame(data = {'code':preproc_scripts, 'problem_num':problem_nums})

code_df.sort_values('problem_num').reset_index()

,index,code,problem_num
0,0,a = [x for x in range(9)]\nfor i in a:\n\tfor ...,problem001
1,96,"for i in range(1, 10):\n\tfor j in range(1, 10...",problem001
2,97,"import itertools\nr = range(1,10)\nfor (x,y) i...",problem001
3,98,i=1\nwhile i<10:\n\tj=1\n\twhile j<10:\n\t\tpr...,problem001
4,99,"i=[1,2,3,4,5,6,7,8,9]\nj=[1,2,3,4,5,6,7,8,9]\n...",problem001
...,...,...,...
45096,45001,import math\ndef factorization(n):\n\tarr = []...,problem300
45097,45002,"a,b=map(int,input().split())\ng=1\nwhile 1:\n ...",problem300
45098,45003,from math import gcd\ndef make_divisors(n):\n\...,problem300
45099,45005,import math\ndef factorization(n):\n\td = [1]\...,problem300


In [ ]:
'''
#'%03d'
i = [i for i in range(1,301)]
code_df['problem_num'].replace('problem{i}', 'i')
'''

SyntaxError: ignored

In [ ]:
'''
a = ['problem001','problem002','problem003','problem004']

for i in range(len(a)):
    i = a.replace('problem{i}', '{i}')
    print(i)
'''

AttributeError: ignored

In [1]:
test_csv = pd.read_csv("/content/drive/MyDrive/code_similarity/open/test.csv")

NameError: ignored

In [ ]:
test_csv.head(10000)

In [ ]:
x = code_df['code']  #.drop('problem_num', axis = 1).values
y = code_df['problem_num']
print(type(x))
print(type(y))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
#test_x = test_csv['code1']
#test_y = test_csv['code2']

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y,
                                                  random_state=42,
                                                  test_size=0.2,
                                                  stratify = y
                                                 )

In [ ]:
print(x_train.shape)
print(x_val.shape)

print(y_train.shape)
print(y_val.shape)
y_train

(36080,)
(9021,)
(36080,)
(9021,)


29826    problem199
33372    problem222
30012    problem200
6227     problem042
32364    problem216
            ...    
6596     problem044
4816     problem033
40695    problem271
39649    problem264
28865    problem192
Name: problem_num, Length: 36080, dtype: object

In [ ]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_val)
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)

"""
x_train = np.array(x_train)
y_train = np.array(y_val)"""

'\nx_train = np.array(x_train)\ny_train = np.array(y_val)'

In [ ]:
'''threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
'''

'threshold = 2\ntotal_cnt = len(tokenizer.word_index) # 단어의 수\nrare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트\ntotal_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합\nrare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합\n\n# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.\nfor key, value in tokenizer.word_counts.items():\n    total_freq = total_freq + value\n\n    # 단어의 등장 빈도수가 threshold보다 작으면\n    if(value < threshold):\n        rare_cnt = rare_cnt + 1\n        rare_freq = rare_freq + value\n\nprint(\'단어 집합(vocabulary)의 크기 :\',total_cnt)\nprint(\'등장 빈도가 %s번 이하인 희귀 단어의 수: %s\'%(threshold - 1, rare_cnt))\nprint("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)\nprint("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)\n'

In [ ]:
'''# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)
'''

"# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.\n# 0번 패딩 토큰을 고려하여 + 1\nvocab_size = total_cnt - rare_cnt + 1\nprint('단어 집합의 크기 :',vocab_size)\n"

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for code_line in nested_list:
    if(len(code_line) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 200
below_threshold_len(max_len, x_train)

전체 샘플 중 길이가 200 이하인 샘플의 비율: 96.77660753880266


In [ ]:
pad_x_train = pad_sequences(x_train, maxlen=max_len)
pad_x_val = pad_sequences(x_val, maxlen=max_len)

In [ ]:
print(len(pad_x_train[0]))
print(len(pad_x_val[0]))

200
200


In [ ]:
# 다중입력 함수형 모델 API
model = Sequential()

embedding_dim = 64

'''input_text = Input(shape=(None,))
embadded_text = layers.Embedding(input_dim = max_len, output_dim = embedding_dim))
x_1 = layers.Dense(32, activation='relu')(input_tensor)
x_2 = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
'''
model.add(Embedding(input_dim = max_len, output_dim = embedding_dim))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='softmax'))
model.add(Flatten())

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          12800     
                                                                 
 lstm_3 (LSTM)               (None, None, 128)         98816     
                                                                 
 lstm_4 (LSTM)               (None, None, 128)         131584    
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
 flatten_1 (Flatten)         (None, 1)                 0         
                                                                 
Total params: 292,673
Trainable params: 292,673
Non-tr

In [ ]:
istory = model.fit(pad_x_train, y_train, 
                    epochs=15,
                    callbacks=[es, mc],
                    batch_size=64
                   )

Epoch 1/15


UnimplementedError: ignored

In [ ]:
# txt 파일 DF 저장
'''
csv_tr = []
csv_tr2 = []
similarity = []
for i in glob.glob("./open/code/*/*.txt"):
    with open(os.path.join(os.getcwd(), i), 'r') as f:
        text = f.read()
        #무작위 행 셔플
        csv_tr.append(text)
        csv_tr2.append(text)
        similarity.append(1)
csv_tr = sklearn.utils.shuffle(csv_tr, random_state = 42)
'''

In [ ]:
# DF 생성
'''
code_csv = pd.DataFrame(zip(csv_tr, csv_tr2,similarity),
                        columns=['code1', 'code2','similarity'])
code_csv.to_csv("code_csv.csv")
'''

In [ ]:
tokenizer = Tokenizer() # 가장 빈도가 높은 1000개의 단어만 선택하도록 Tokenizer 객체를 만든다.
tokenizer.fit_on_texts(code_csv['code1'],code_csv['code2']) # 단어 인덱스 구축
sequences = tokenizer.texts_to_sequences(code_csv['code1']) # 문자열을 정수 인덱스의 리스트로 변환한다. 

one_hot_results = tokenizer.texts_to_matrix(code_csv['code1'], mode='binary') # 직접 원-핫 이진 벡터 표현을 얻을 수 있다.
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1

In [ ]:
print('단어 집합 :',vocab_size)

In [ ]:
X_encoded = tokenizer.texts_to_sequences(code_csv['code1'])
print('정수 인코딩 결과 :',sequences)

In [ ]:
vocab_size = 34668
output_dim = 128

v = Embedding(vocab_size, output_dim)